In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


url = 'https://raw.githubusercontent.com/TheLazyCactus/ML_Project/refs/heads/main/ML_Project_safety.csv'
df = pd.read_csv(url, sep=";", low_memory =False)

Need to change the order to get the oldest value first

In [2]:
df = df.sort_values(by="Year", ascending=True)
#Drop columns
cols_to_remove = ["FAR", "TRIR total","TRIR company only", "TRIR contractor only", "LTIR company only", "LTIR contractor only"]
df.drop(["FAR total", "LTIR company only","LTIR contractor only","TRIR company only", "TRIR contractor only"], axis = 1, inplace=True)

df = df.drop(columns=[col for col in cols_to_remove if col in df.columns])  # List of columns to convert
cols = ["LTIR total"]
cols_to_process = [col for col in cols if col in df.columns]

# Replace commas with dots and convert to float for the existing columns
df[cols_to_process] = df[cols_to_process].replace(',', '.', regex=True).astype(float)
df[cols] = df[cols].astype(float)

df = df.fillna(0)

EDA

In [ ]:
print(df.shape)
print(df.columns)
print(df.dtypes)
print(df.nunique())
print(df.isna().sum())

Filter to keep only the last 3 years

In [3]:
from sklearn.preprocessing import OneHotEncoder

# Ensure the "Year" column exists and filter only the last 3 years
year_column = "Year"  # Adjust this if your column name is different
if year_column not in df.columns:
    raise KeyError(f"Column '{year_column}' not found. Available columns: {df.columns}")

df_filtered = df[df[year_column].isin([2020, 2021, 2022, 2023])]

# Display filtered data
print(df_filtered.head())

     Year Company code  LTIR total
167  2020   N                 0.41
168  2020   O                 0.39
169  2020   P                 0.37
170  2020   Q                 0.34
173  2020   T                 0.26


Lag creation

In [4]:
# Create lag features: LTIR from the past 1 2 and 3 years
df_filtered['LTIR_1'] = df_filtered.groupby('Company code')['LTIR total'].shift(1)
df_filtered['LTIR_2'] = df_filtered.groupby('Company code')['LTIR total'].shift(2)
df_filtered['LTIR_3'] = df_filtered.groupby('Company code')['LTIR total'].shift(3)
df_filtered = df_filtered.rename(columns={"LTIR total": "LTIR 2023", "LTIR_1": "LTIR 2022", "LTIR_2": "LTIR 2021", "LTIR_3": "LTIR 2020"})
# Drop rows with NaN values (first 2 years for each company)
data = df_filtered.dropna().reset_index(drop=True)
print(data)  # Fixed column name

# Drop rows with NaN values due to shifting
df_filtered = df_filtered.dropna().reset_index(drop=True)  # Ensure you are dropping from df_final, not df

    Year Company code  LTIR 2023  LTIR 2022  LTIR 2021  LTIR 2020
0   2023   Q                0.51       0.44       0.38       0.34
1   2023   R                0.51       0.34       0.38       0.31
2   2023   Overall          0.24       0.28       0.22       0.22
3   2023   W                0.30       0.26       0.23       0.25
4   2023   X                0.28       0.24       0.23       0.24
5   2023   Y                0.27       0.23       0.21       0.24
6   2023   P                0.51       0.45       0.40       0.37
7   2023   V                0.33       0.27       0.27       0.25
8   2023   O                0.54       0.53       0.41       0.39
9   2023   C                1.22       1.08       0.99       1.04
10  2023   M                0.55       0.56       0.45       0.44
11  2023   L                0.59       0.58       0.46       0.47
12  2023   K                0.63       0.63       0.66       0.50
13  2023   J                0.67       0.70       0.68       0.50
14  2023  

C:\Users\UTILISATEUR\AppData\Local\Temp\ipykernel_25748\3061365396.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['LTIR_1'] = df_filtered.groupby('Company code')['LTIR total'].shift(1)
C:\Users\UTILISATEUR\AppData\Local\Temp\ipykernel_25748\3061365396.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['LTIR_2'] = df_filtered.groupby('Company code')['LTIR total'].shift(2)
C:\Users\UTILISATEUR\AppData\Local\Temp\ipykernel_25748\3061365396.py:4: SettingWithCopyWarning: 
A value is

One hot encoding

In [5]:
from sklearn.preprocessing import OneHotEncoder


# Ensure "Company code" exists before encoding
company_column = "Company code"
if company_column not in df_filtered.columns:
    raise KeyError(f"Column '{company_column}' not found. Available columns: {df_filtered.columns}")

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Fit-transform the 'Company code' column
encoded_company = encoder.fit_transform(df_filtered[[company_column]])

# Get feature names for encoded categories
company_columns = encoder.get_feature_names_out([company_column])

# Convert to DataFrame
df_encoded = pd.DataFrame(encoded_company, columns=company_columns)

# Merge encoded data back into df_filtered and drop the original "Company code"
df_final = pd.concat([df_filtered.reset_index(drop=True), df_encoded], axis=1)
df_final.drop(columns=[company_column], inplace=True, errors='ignore')

import pickle
with open('encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)


In [6]:
df_final.drop(columns=["Year"], inplace=True)

**Model training**

In [7]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Check that df_final has the expected columns
required_columns = ['LTIR 2020', 'LTIR 2021', 'LTIR 2022', 'LTIR 2023']
missing_columns = [col for col in required_columns if col not in df_final.columns]
if missing_columns:
    raise ValueError(f"Missing columns: {missing_columns}")

# Features for initial model training
X = df_final[['LTIR 2020', 'LTIR 2021', 'LTIR 2022']]  # Predictors
y = df_final['LTIR 2023']  # Target variable

# Train Linear Regression model
model = LinearRegression()
linear_reg = model.fit(X, y)

# Prepare storage for future predictions
future_data = []

# Iterate through each row in the dataframe to predict future LTIR
for _, row in df_final.iterrows():
    # Extract relevant lag features
    ltir_2021 = row['LTIR 2021']
    ltir_2022 = row['LTIR 2022']
    ltir_2023 = row['LTIR 2023']

    # Predict 2024 LTIR using LTIR 2021, LTIR 2022, and LTIR 2023
    input_2024 = pd.DataFrame([[ltir_2021, ltir_2022, ltir_2023]], columns=['LTIR 2020', 'LTIR 2021', 'LTIR 2022'])
    predicted_2024 = linear_reg.predict(input_2024)[0]

    # Predict 2025 LTIR using LTIR 2022, LTIR 2023, and predicted 2024
    input_2025 = pd.DataFrame([[ltir_2022, ltir_2023, predicted_2024]], columns=['LTIR 2020', 'LTIR 2021', 'LTIR 2022'])
    predicted_2025 = linear_reg.predict(input_2025)[0]

    # Predict 2026 LTIR using LTIR 2023, predicted 2024, and predicted 2025
    input_2026 = pd.DataFrame([[ltir_2023, predicted_2024, predicted_2025]], columns=['LTIR 2020', 'LTIR 2021', 'LTIR 2022'])
    predicted_2026 = linear_reg.predict(input_2026)[0]

    # Append predictions to the future_data list
    future_data.append([predicted_2024, predicted_2025, predicted_2026])

# Convert predictions into a DataFrame
future_df = pd.DataFrame(future_data, columns=['LTIR 2024', 'LTIR 2025', 'LTIR 2026'])

# Concatenate predictions with original dataset
df_final = pd.concat([df_final, future_df], axis=1)

# Display the updated DataFrame
print("Updated DataFrame with future LTIR values:")
print(df_final)


Updated DataFrame with future LTIR values:
    LTIR 2023  LTIR 2022  LTIR 2021  LTIR 2020  Company code_A           \
0        0.51       0.44       0.38       0.34                      0.0   
1        0.51       0.34       0.38       0.31                      0.0   
2        0.24       0.28       0.22       0.22                      0.0   
3        0.30       0.26       0.23       0.25                      0.0   
4        0.28       0.24       0.23       0.24                      0.0   
5        0.27       0.23       0.21       0.24                      0.0   
6        0.51       0.45       0.40       0.37                      0.0   
7        0.33       0.27       0.27       0.25                      0.0   
8        0.54       0.53       0.41       0.39                      0.0   
9        1.22       1.08       0.99       1.04                      0.0   
10       0.55       0.56       0.45       0.44                      0.0   
11       0.59       0.58       0.46       0.47           

In [8]:
df_final.columns

Index(['LTIR 2023', 'LTIR 2022', 'LTIR 2021', 'LTIR 2020',
       'Company code_A         ', 'Company code_AA        ',
       'Company code_B         ', 'Company code_BB        ',
       'Company code_C         ', 'Company code_CC        ',
       'Company code_D         ', 'Company code_DD        ',
       'Company code_E         ', 'Company code_EE        ',
       'Company code_F         ', 'Company code_FF        ',
       'Company code_G         ', 'Company code_H         ',
       'Company code_HH        ', 'Company code_I         ',
       'Company code_II        ', 'Company code_J         ',
       'Company code_K         ', 'Company code_KK        ',
       'Company code_L         ', 'Company code_LL        ',
       'Company code_M         ', 'Company code_MM        ',
       'Company code_N         ', 'Company code_NN        ',
       'Company code_O         ', 'Company code_OO        ',
       'Company code_Overall   ', 'Company code_P         ',
       'Company code_PP   

In [9]:
df_final.to_csv("final_LTIR_predictions.csv", index=False)